In [1]:
import numpy as np
from MMOE import MMoE
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve
import keras
from sklearn.model_selection import StratifiedKFold
from keras import Input, Model
from keras.layers import Dense, Concatenate, Lambda, Dropout, Add, Multiply, AveragePooling1D
from keras.utils import plot_model
import random
import sklearn.metrics as metrics
from keras.callbacks import ReduceLROnPlateau
from keras import backend as K
from keras.initializers import VarianceScaling
from keras.utils.vis_utils import model_to_dot
from keras import Model
from sklearn.ensemble import ExtraTreesClassifier
from itertools import combinations
from M_MMOE import M_MMOE_with_pheno
import os

Using TensorFlow backend.


In [2]:
os.environ['PYTHONHASHSEED']=str(42)
random.seed(42)
np.random.seed(42)

### load data

In [3]:
# load fmri data, labels and phenotypic infromation.
ASD_feat = np.load('Data/ASD/ASD_FC_feat_190.npy')
ASD_labels = np.load('Data/ASD/ASD_label_190.npy')
ADHD_feat = np.load('Data/ADHD/ADHD_whole_FC_feat.npy')
ADHD_labels = np.load('Data/ADHD/ADHD_label_whole.npy')
COBRE_feat = np.load('Data/SCZ/cobre_FC_feat_190.npy')
COBRE_labels = np.load('Data/SCZ/cobre_label.npy')
ASD_pheno = np.load('Data/ASD/ASD_phenotypic_data_whole.npy').T
ADHD_pheno = np.load('Data/ADHD/ADHD_phenotypic_data_whole.npy').T
COBRE_pheno = np.load('Data/SCZ/COBRE_phenotypic_data_whole.npy').T

# load selected connectivity features and cluster index
selected_feature_index = np.load('Data/selected_feature_index.npy',allow_pickle=True)
cluster_indices = np.load('Data/cluster_indices.npy',allow_pickle=True)

In [4]:
label = np.hstack((ASD_labels, ADHD_labels, COBRE_labels))
feat = np.vstack((ASD_feat, ADHD_feat, COBRE_feat))
pheno = np.vstack((ASD_pheno, ADHD_pheno, COBRE_pheno))

### data spliting

In [5]:
def split_tra_val_set(train_index, test_index):
    length = len(train_index) + len(test_index)
    ASD_num = 1035
    ADHD_num = 939
    x1_train_list = []
    x2_train_list = []
    x3_train_list = []
    x1_test_list = []
    x2_test_list = []
    x3_test_list = []

    for i in range(length):
        if i in train_index:  # training_set
            if i < ASD_num:  # split to x1_train, y1_train
                x1_train_list.append(i)
            if ASD_num <= i < ADHD_num + ASD_num:  # split to x2_train, y2_train
                x2_train_list.append(i)
            if i >= ADHD_num + ASD_num:
                x3_train_list.append(i)
        else:  # testing_test
            if i < ASD_num:  # split to x1_test, y1_test
                x1_test_list.append(i)
            if ASD_num <= i < ADHD_num + ASD_num:  # split to x2_test, y2_test
                x2_test_list.append(i)
            if i >= ADHD_num + ASD_num:  # split to x2_test, y2_test
                x3_test_list.append(i)
    return x1_train_list, x2_train_list, x3_train_list, x1_test_list, x2_test_list, x3_test_list

In [6]:
# relabeling
relabel_ADHD = []
for i in range(len(ADHD_labels)):
    if ADHD_labels[i] == 1:
        relabel_ADHD.append(2)
    else:
        relabel_ADHD.append(3)

relabel_COBRE = []
for i in range(len(COBRE_labels)):
    if COBRE_labels[i] == 1:
        relabel_COBRE.append(4)
    else:
        relabel_COBRE.append(5)

stratify_labels = np.hstack((ASD_labels, np.array(relabel_ADHD), np.array(relabel_COBRE)))
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

### traning M-MMOE

In [19]:
ASD_MMOE_acc = []
ADHD_MMOE_acc = []
COBRE_MMOE_acc = []
for train_index, test_index in skf.split(feat, stratify_labels):
    K.clear_session()
    (x1_train_list, x2_train_list, x3_train_list, x1_test_list, x2_test_list, x3_test_list) = split_tra_val_set(
        train_index, test_index)
    x1_train = feat[x1_train_list]
    y1_train = label[x1_train_list]
    x1_train_pheno = pheno[x1_train_list]

    x2_train = feat[x2_train_list]
    y2_train = label[x2_train_list]
    x2_train_pheno = pheno[x2_train_list]

    x3_train = feat[x3_train_list]
    y3_train = label[x3_train_list]
    x3_train_pheno = pheno[x3_train_list]

    x1_test = feat[x1_test_list]
    y1_test = label[x1_test_list]
    x1_test_pheno = pheno[x1_test_list]

    x2_test = feat[x2_test_list]
    y2_test = label[x2_test_list]
    x2_test_pheno = pheno[x2_test_list]

    x3_test = feat[x3_test_list]
    y3_test = label[x3_test_list]
    x3_test_pheno = pheno[x3_test_list]

    list_train1 = [0 for i in range(x2_train.shape[0])]
    list_train2 = [0 for i in range(x3_train.shape[0])]
    y_ASD_train = np.hstack([y1_train, np.array(list_train1), np.array(list_train2)])

    list_train1 = [0 for i in range(x1_train.shape[0])]
    list_train2 = [0 for i in range(x3_train.shape[0])]
    y_ADHD_train = np.hstack([np.array(list_train1), y2_train, np.array(list_train2)])

    list_train1 = [0 for i in range(x1_train.shape[0])]
    list_train2 = [0 for i in range(x2_train.shape[0])]
    y_COBRE_train = np.hstack([np.array(list_train1), np.array(list_train2), y3_train])

    X_train = np.vstack([x1_train, x2_train, x3_train])
    x1_train_pheno = np.vstack([x1_train_pheno, x2_train_pheno, x3_train_pheno])
    y_train = [y_ASD_train, y_ADHD_train, y_COBRE_train]

    # shuffle
    index = [i for i in range(X_train.shape[0])]
    random.shuffle(index)

    X_train = X_train[index]
    X_train_pheno = x1_train_pheno[index]
    y_ASD_train = y_ASD_train[index]
    y_ADHD_train = y_ADHD_train[index]
    y_COBRE_train = y_COBRE_train[index]
    y_train = [y_ASD_train, y_ADHD_train, y_COBRE_train]

    mmoe_units = 100
    experts_num = 6
    tasks_num = 3

    print(X_train.shape)
    # record
    ASD_acc, ADHD_acc, COBRE_acc = M_MMOE_with_pheno(X_train, y_train, X_train_pheno, x1_test,
                                                                   y1_test, x1_test_pheno,
                                                                   x2_test, y2_test, x2_test_pheno, x3_test,
                                                                   y3_test, x3_test_pheno,
                                                                   cluster_indices, selected_feature_index, 0.5,
                                                                   mmoe_units, experts_num, tasks_num)
    ASD_MMOE_acc.append(ASD_acc)
    ADHD_MMOE_acc.append(ADHD_acc)
    COBRE_MMOE_acc.append(COBRE_acc)

print("ASD mean accuracy: ", np.mean(ASD_MMOE_acc))
print("ADHD mean accuracy: ", np.mean(ADHD_MMOE_acc))
print("COBRE mean accuracy: ", np.mean(COBRE_MMOE_acc))

(1904, 17955)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input0 (InputLayer)        (None, 932)          0                                            
__________________________________________________________________________________________________
main_input1 (InputLayer)        (None, 882)          0                                            
__________________________________________________________________________________________________
main_input2 (InputLayer)        (None, 670)          0                                            
__________________________________________________________________________________________________
main_input3 (InputLayer)        (None, 699)          0                                            
______________________________________________________________________________